# 零基础实战机器学习

## 第16讲 会员流失情况评估

作者 黄佳

极客时间专栏链接：https://time.geekbang.org/column/intro/438


问题：判断易速鲜花会员的是否会流失情况

易速鲜花公司拥有多年的会员记录，以及会员停止续费的情况。

通过逻辑回归和神经网络等机器学习模型，我们可以判断出客户是否离开，这是一个典型的的二元分类问题。

这一讲我们集中介绍分类问题的评估指标：混淆矩阵，精确率，召回率，F1分数，ROC曲线和AUC。


## 数据的读入和预处理

In [ ]:
# 导入数据包
import numpy as np # 
import pandas as pd #
import matplotlib.pyplot as plt #

In [ ]:
df_member = pd.read_csv('易速鲜花会员留存.csv')
df_member

### 数据清洗

In [ ]:
#把总消费字段转换成数值字段
df_member['总消费'] = pd.to_numeric(df_member['总消费'], errors='coerce')
df_member['总消费'].fillna(0, inplace=True)

### 数据可视化

In [ ]:
plt.figure(figsize=(6,4))
ax = df_member.groupby('已停付会费').count()['用户码'].plot.pie(autopct='%1.0f%%') #饼图
plt.xlabel('是否流失')
plt.show() #显示

### 特征工程

In [ ]:
df_member['已停付会费'].replace(to_replace='是', value=1, inplace=True) #流失-1
df_member['已停付会费'].replace(to_replace='否',  value=0, inplace=True) #未流失-0

In [ ]:
df_member['性别'].replace(to_replace='女', value=0, inplace=True) #女生-0
df_member['性别'].replace(to_replace='男', value=1, inplace=True) #男生-1

In [ ]:
# 字段中'Yes' or 'No'转换成为模型可以读取的数值,（布尔型数据，也是数值数据）
binary_features = ['玫瑰套餐', '紫罗兰套餐', '郁金香套餐', '百合套餐', '康乃馨套餐', '胡姬花套餐', 
                   '生日套餐','情人节套餐']
for field in binary_features:
    df_member[field] = df_member[field] == '是'

In [ ]:
df_member

### 数据整理
先做数据整理工作，把每个数据字段都转换为可以处理的字段

In [ ]:
# One hot encode 分类字段
category_features = ['会员卡类型', '会费支付方式']
df_member = pd.get_dummies(df_member, drop_first=True, columns=category_features)
df_member

### 构建特征集和标签集

In [ ]:
X = df_member.drop(['用户码','已停付会费'], axis = 1) # 构建特征集，用户吗字段属于无用特征
y = df_member.已停付会费.values # 构建标签集

### 拆分数据集

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 4)

## 选择算法

这里我们比较逻辑回归和神经网络两种算法

### 逻辑回归模型

#### 导入模型

In [ ]:
from sklearn.linear_model import LogisticRegression #导入逻辑回归模型
logreg = LogisticRegression(solver='lbfgs', max_iter=1000) # logreg,就代表是逻辑回归模型

#### 训练机器

In [ ]:
logreg.fit(X_train,y_train) # fit,就相当于是梯度下降

#### 评估分数

In [ ]:
print("SK-learn逻辑回归预测准确率{:.2f}%".format(logreg.score(X_test,y_test)*100))

#### 预测结果

In [ ]:
y_pred_logreg = logreg.predict(X_test)
print("逻辑回归对测试集第一个用户的预测结果", y_pred_logreg[0])

#### 混淆矩阵

In [ ]:
from sklearn.metrics import confusion_matrix # 导入混淆矩阵
import seaborn as sns #导入seaborn画图工具箱
def show_matrix(y_test, y_pred, label): # 定义一个函数显示混淆矩阵
    cm = confusion_matrix(y_test,y_pred) # 调用混淆矩阵
    plt.title(label) # 标题
    sns.heatmap(cm,annot=True,cmap="Blues",fmt="d",cbar=False) # 热力图设定
    plt.show() # 显示混淆矩阵

In [ ]:
show_matrix(y_test, y_pred_logreg,label='混淆矩阵（逻辑回归)') # 逻辑回归

#### 分类报告

In [ ]:
from sklearn.metrics import classification_report # 导入分类报告
def show_report(X_test, y_test, y_pred): # 定义一个函数显示分类报告
    # print (y_pred)
    #np.set_printoptions(threshold=np.inf)
    # print (np.where(predictions > 0.5, 0, predictions))
#     if y_test.shape != (2000,1):
#         y_test = y_test.values # 把Panda series转换成Numpy array
#         y_test = y_test.reshape((len(y_test),1)) # 转换成与y_pred相同的形状 
    #target_names = [str(x) for x in lb.classes_]
    print(classification_report(y_test,y_pred,labels=[0, 1])) #打印分类报告  

In [ ]:
show_report(X_test, y_test, y_pred_logreg)

### 神经网络模型

In [ ]:
#!pip install keras
#!pip install tensorflow

In [ ]:
import keras # 导入Keras库
from keras.models import Sequential # 导入Keras序贯模型
from keras.layers import Dense # 导入Keras密集连接层
dnn = Sequential() # 创建一个序贯DNN模型
dnn.add(Dense(units=12, input_dim=17, activation = 'relu')) # 添加输入层
dnn.add(Dense(units=24, activation = 'relu')) # 添加隐层
dnn.add(Dense(units=1, activation = 'sigmoid')) # 添加输出层
dnn.summary() # 显示网络模型（这个语句不是必须的）
# 编译神经网络，指定优化器，损失函数，以及评估标准
dnn.compile(optimizer = 'RMSProp', loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
X_train.shape

In [ ]:
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

In [ ]:
history = dnn.fit(X_train, y_train, # 指定训练集
                  epochs=30,        # 指定训练的轮次
                  batch_size=64,    # 指定数据批量
                  validation_split=0.2) #这里直接从训练集数据中拆分验证集，更方便

In [ ]:
def show_history(history): # 显示训练过程中的学习曲线
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.figure(figsize=(12,4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    plt.subplot(1, 2, 2)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show() 

In [ ]:
show_history(history) # 调用这个函数

In [ ]:
result = dnn.evaluate(X_test, y_test) #评估测试集上的准确率
print('DNN的测试准确率为',"{0:.2f}%".format(result[1]))

In [ ]:
prediction = dnn.predict(X_test) #预测测试集的图片分类
print('第一个用户分类结果为:', np.argmax(prediction[0]))

In [ ]:
y_pred = dnn.predict(X_test,batch_size=10) # 预测测试集的标签
y_pred_dnn = np.round(y_pred) # 将分类概率值转换成0/1整数值
show_matrix(y_test, y_pred_dnn, label='混淆矩阵（神经网络归一化之前）') # 混淆矩阵（神经网络归一化之前）

In [ ]:
show_report(X_test, y_test, y_pred_dnn)

### 神经网络模型-归一化之后

In [ ]:
from sklearn.preprocessing import MinMaxScaler #导入归一化缩放器
scaler = MinMaxScaler() #创建归一化缩放器
X_train = scaler.fit_transform(X_train) #拟合并转换训练集数据
X_test = scaler.transform(X_test) #转换测试集数据

In [ ]:
history = dnn.fit(X_train, y_train, # 指定训练集
                  epochs=30,        # 指定训练的轮次
                  batch_size=64,    # 指定数据批量
                  validation_split=0.2) #指定验证集,这里为了简化模型，直接用训练集数据
show_history(history) # 调用这个函数

In [ ]:
result = dnn.evaluate(X_test, y_test) #评估测试集上的准确率
print('DNN（归一化之后）的测试准确率为',"{0:.2f}%".format(result[1]))

In [ ]:
y_pred = dnn.predict(X_test,batch_size=10) # 预测测试集的标签
y_pred_dnn_scale = np.round(y_pred) # 将分类概率值转换成0/1整数值
show_matrix(y_test, y_pred_dnn_scale, label='混淆矩阵（神经网络归一化之后）') # 混淆矩阵（神经网络归一化之后）

In [ ]:
show_report(X_test, y_test, y_pred_dnn_scale)

#### ROC曲线和AUC

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:
y_pred_logreg = logreg.predict_proba(X_test)[:, 1]
fpr_logreg, tpr_logreg, thresholds_logreg = roc_curve(y_test, y_pred_logreg)
auc_logreg = auc(fpr_logreg, tpr_logreg)

In [ ]:
y_pred_dnn_scale = dnn.predict(X_test).ravel()
fpr_dnn_scale, tpr_dnn_scale, thresholds_dnn_scale = roc_curve(y_test, y_pred_dnn_scale)
auc_dnn_scale = auc(fpr_dnn_scale, tpr_dnn_scale)

In [ ]:
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_logreg, tpr_logreg, label='逻辑回归 (area = {:.3f})'.format(auc_logreg))
plt.plot(fpr_dnn_scale, tpr_dnn_scale, label='神经网络 (area = {:.3f})'.format(auc_dnn_scale))
plt.xlabel('False Positive Rate')
plt.ylabel('True Rositive Rate')
plt.title('ROC 曲线')
plt.legend(loc='best')
plt.show()